In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import MobileNetV2
from keras.applications.mobilenetv2 import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 35534421
EPOCHS = 150
size = 64
batchsize = 640
lw = 6
channel = 3
STEPS = TOT / EPOCHS / batchsize
NCATS = 340
mixup = 0
center = False
optimizer = 'adam'
cb = 'raw'

In [3]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, 
                               channel=channel, center = center)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

x shape (34000, 64, 64, 3) x max 255
(34000, 64, 64, 3) (34000, 340)
Validation array memory 1.56 GB


In [4]:
K.clear_session()
# model = Xception(input_shape=(size, size, channel), weights=None, classes=NCATS)
model = MobileNetV2(input_shape=(size,size,channel), weights=None, classes=NCATS)

if optimizer == 'sgd':
    opt = SGD(0.002, momentum=0.9, nesterov=True)
    opt = NormalizedOptimizer(opt, normalization='l2')
elif optimizer == 'adam':
    opt = Adam(lr=0.002)

model.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 65, 65, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 32, 32, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 32, 32, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [5]:
train_datagen = image_generator_xd(df_path = '../input/train_all_balance.csv', 
                                   size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel, mixup=mixup, center=center)

In [6]:
model_prefix = 'mobile{}_linelength_lw{}_balance_{}_{}'.format(size, lw, mixup, optimizer, cb)
if center:
    model_prefix += '_center'
if channel == 3:
    model_prefix += '_parts'
print(model_prefix)
check_path = "./models/{}.model".format(model_prefix)
fold = 9
if cb == 'raw':
    callbks = [
        ReduceLROnPlateau(monitor='val_top_3_accuracy', factor=0.5, patience=5,
                          min_delta=0.005, mode='max', cooldown=3, verbose=1),
        callbacks.ModelCheckpoint(check_path,
                                    monitor='val_top_3_accuracy', 
                                    mode = 'max', save_best_only=True, verbose=1),
        callbacks.TensorBoard(log_dir='./log/{}'.format(model_prefix)),
        EarlyStopping(monitor='val_top_3_accuracy', mode='max',patience=20, verbose=1)
    ]
elif cb == 'snap':
    snapshot = SnapshotCallbackBuilder(nb_epochs=EPOCHS,
                                       nb_snapshots=1,
                                       size = size,
                                       init_lr=1e-3, 
                                       fold=fold)

    callbks = snapshot.get_callbacks(model_prefix = model_prefix)

mobile64_linelength_lw6_balance_0_adam_parts


In [ ]:
hists = []
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/150
371/370 [==============================] - 128s 345ms/step - loss: 3.7383 - categorical_crossentropy: 3.7383 - categorical_accuracy: 0.2116 - top_3_accuracy: 0.3644 - val_loss: 14.5596 - val_categorical_crossentropy: 14.5596 - val_categorical_accuracy: 0.0105 - val_top_3_accuracy: 0.0292

Epoch 00001: val_top_3_accuracy improved from -inf to 0.02924, saving model to ./models/mobile64_linelength_lw6_balance_0_adam_parts.model
Epoch 2/150
371/370 [==============================] - 120s 323ms/step - loss: 2.2188 - categorical_crossentropy: 2.2188 - categorical_accuracy: 0.4687 - top_3_accuracy: 0.6806 - val_loss: 13.2788 - val_categorical_crossentropy: 13.2788 - val_categorical_accuracy: 0.0377 - val_top_3_accuracy: 0.0772

Epoch 00002: val_top_3_accuracy improved from 0.02924 to 0.07721, saving model to ./models/mobile64_linelength_lw6_balance_0_adam_parts.model
Epoch 3/150
371/370 [==============================] - 120s 323ms/step - loss: 1.8708 - categorical_crossentropy: 1

371/370 [==============================] - 120s 323ms/step - loss: 1.1415 - categorical_crossentropy: 1.1415 - categorical_accuracy: 0.7103 - top_3_accuracy: 0.8721 - val_loss: 5.5947 - val_categorical_crossentropy: 5.5947 - val_categorical_accuracy: 0.2214 - val_top_3_accuracy: 0.3833

Epoch 00021: val_top_3_accuracy did not improve from 0.49397
Epoch 22/150
371/370 [==============================] - 120s 322ms/step - loss: 1.1291 - categorical_crossentropy: 1.1291 - categorical_accuracy: 0.7127 - top_3_accuracy: 0.8740 - val_loss: 4.2982 - val_categorical_crossentropy: 4.2982 - val_categorical_accuracy: 0.3236 - val_top_3_accuracy: 0.5266

Epoch 00022: val_top_3_accuracy improved from 0.49397 to 0.52665, saving model to ./models/mobile64_linelength_lw6_balance_0_adam_parts.model
Epoch 23/150
371/370 [==============================] - 120s 323ms/step - loss: 1.1183 - categorical_crossentropy: 1.1183 - categorical_accuracy: 0.7150 - top_3_accuracy: 0.8765 - val_loss: 4.5846 - val_categ

371/370 [==============================] - 120s 323ms/step - loss: 0.9599 - categorical_crossentropy: 0.9599 - categorical_accuracy: 0.7543 - top_3_accuracy: 0.8985 - val_loss: 1.3046 - val_categorical_crossentropy: 1.3046 - val_categorical_accuracy: 0.6806 - val_top_3_accuracy: 0.8538

Epoch 00042: val_top_3_accuracy improved from 0.80544 to 0.85376, saving model to ./models/mobile64_linelength_lw6_balance_0_adam_parts.model
Epoch 43/150
371/370 [==============================] - 120s 323ms/step - loss: 0.9575 - categorical_crossentropy: 0.9575 - categorical_accuracy: 0.7560 - top_3_accuracy: 0.8979 - val_loss: 1.1935 - val_categorical_crossentropy: 1.1935 - val_categorical_accuracy: 0.7030 - val_top_3_accuracy: 0.8672

Epoch 00043: val_top_3_accuracy improved from 0.85376 to 0.86721, saving model to ./models/mobile64_linelength_lw6_balance_0_adam_parts.model
Epoch 44/150
371/370 [==============================] - 120s 323ms/step - loss: 0.9405 - categorical_crossentropy: 0.9405 - cat

371/370 [==============================] - 120s 323ms/step - loss: 0.8872 - categorical_crossentropy: 0.8872 - categorical_accuracy: 0.7728 - top_3_accuracy: 0.9082 - val_loss: 0.8891 - val_categorical_crossentropy: 0.8891 - val_categorical_accuracy: 0.7714 - val_top_3_accuracy: 0.9077

Epoch 00061: val_top_3_accuracy improved from 0.90724 to 0.90768, saving model to ./models/mobile64_linelength_lw6_balance_0_adam_parts.model
Epoch 62/150
371/370 [==============================] - 120s 322ms/step - loss: 0.8843 - categorical_crossentropy: 0.8843 - categorical_accuracy: 0.7735 - top_3_accuracy: 0.9085 - val_loss: 0.8888 - val_categorical_crossentropy: 0.8888 - val_categorical_accuracy: 0.7711 - val_top_3_accuracy: 0.9075

Epoch 00062: val_top_3_accuracy did not improve from 0.90768
Epoch 63/150
371/370 [==============================] - 120s 324ms/step - loss: 0.8811 - categorical_crossentropy: 0.8811 - categorical_accuracy: 0.7738 - top_3_accuracy: 0.9090 - val_loss: 0.8965 - val_categ

371/370 [==============================] - 120s 324ms/step - loss: 0.8485 - categorical_crossentropy: 0.8485 - categorical_accuracy: 0.7820 - top_3_accuracy: 0.9137 - val_loss: 0.8379 - val_categorical_crossentropy: 0.8379 - val_categorical_accuracy: 0.7854 - val_top_3_accuracy: 0.9138

Epoch 00081: val_top_3_accuracy improved from 0.91376 to 0.91382, saving model to ./models/mobile64_linelength_lw6_balance_0_adam_parts.model
Epoch 82/150
371/370 [==============================] - 120s 323ms/step - loss: 0.8499 - categorical_crossentropy: 0.8499 - categorical_accuracy: 0.7822 - top_3_accuracy: 0.9138 - val_loss: 0.8369 - val_categorical_crossentropy: 0.8369 - val_categorical_accuracy: 0.7855 - val_top_3_accuracy: 0.9137

Epoch 00082: val_top_3_accuracy did not improve from 0.91382
Epoch 83/150
371/370 [==============================] - 120s 323ms/step - loss: 0.8422 - categorical_crossentropy: 0.8422 - categorical_accuracy: 0.7829 - top_3_accuracy: 0.9141 - val_loss: 0.8362 - val_categ

371/370 [==============================] - 120s 324ms/step - loss: 0.8495 - categorical_crossentropy: 0.8495 - categorical_accuracy: 0.7822 - top_3_accuracy: 0.9128 - val_loss: 0.8314 - val_categorical_crossentropy: 0.8314 - val_categorical_accuracy: 0.7870 - val_top_3_accuracy: 0.9150

Epoch 00101: val_top_3_accuracy did not improve from 0.91512
Epoch 102/150
371/370 [==============================] - 120s 324ms/step - loss: 0.8425 - categorical_crossentropy: 0.8425 - categorical_accuracy: 0.7825 - top_3_accuracy: 0.9143 - val_loss: 0.8312 - val_categorical_crossentropy: 0.8312 - val_categorical_accuracy: 0.7876 - val_top_3_accuracy: 0.9147

Epoch 00102: val_top_3_accuracy did not improve from 0.91512
Epoch 103/150
371/370 [==============================] - 120s 324ms/step - loss: 0.8413 - categorical_crossentropy: 0.8413 - categorical_accuracy: 0.7840 - top_3_accuracy: 0.9143 - val_loss: 0.8313 - val_categorical_crossentropy: 0.8313 - val_categorical_accuracy: 0.7873 - val_top_3_accu

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel, center = center)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}


In [ ]:
def doodle_predict(model, model_path, x_test):
    model.load_weights(model_path)

    test_predictions = model.predict(x_test, batch_size=128, verbose=1)
    top3 = preds2catids(test_predictions)
    top3cats = top3.replace(id2cat)
    test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
    submission = test[['key_id', 'word']]

    import kaggle_util
    kaggle_util.save_result(submission,  
                            '../result/{}.csv'.format(model_prefix), 
                            'quickdraw-doodle-recognition', 
                            send=True, index=False)

In [ ]:
# doodle_predict(model, check_path, x_test)